# Formulation of policy for given Gird World problem

![state transtion probabilities](StateTransitionProbabilities.png "State Transition Probabilities")

- When two optimal directions are possible, both directions are chosen with a probability **0.5**; Else optimal direction is chosen with probability **1**.
- Given optimal direction is chosen,
    - Probability of moving in desired direction = **0.7**
    - All other non-optimal directions are equi-probable = **0.1**

#### Sample Calculations
##### State-1
- `P(T/s1)`
    
    `= P(Left/s1) * P(T/s1, Left)`  
    
    `= 1 * 0.7  = 0.7`  # Since only one optimal direction
    
    
- `P(s1/s1) = P(s2/s1) = P(s5/s1) = 0.1` # All non-optimal actions are equiprobable and `P(next-state/ non-optimal action) = 1`

Similarly for states s2, s4, s7, s8, s11, s13, 14

##### State-3
- `P(s2/s3)`

    `= (P(Left/s3) * P(s2/ s3, Left)) + (P(Down/s3) * P(s2/ s3, Down))`  # Since 2 equi-probable optimal actions
    
    `= (0.5 * 0.7) + (0.5 * 0.1) = 0.35 + 0.05 = 0.4`
  
- `P(s7/s3)`

    `= (P(Down/s3) * P(s7/ s3, Down)) + (P(Left/s3) * P(s7/ s3, Left))`
    
    `= (0.5 * 0.7) + (0.5 * 0.1) = 0.35 + 0.05 = 0.4` # Same reason as above
    
- `P(s3/s3) = 0.2`  #  Remaining probability

Similarly for state s12

##### State-5
- Using same 2 equi-probable optimal action logic as in state s3,
    
    `P(s4/s5) = P(s1/s5) = 0.4`
    
- Other two states take equal probability in remaining probability as,
    
    `P(s6/s5) = P(s9/s5) = 0.1`
    
Similarly for states s6, s9, s10

In [1]:
# Above policy in the form of dictionary
policy = {
    "s1":[("X",0.7),("s1",0.1),("s2",0.1),("s5",0.1)],
    "s2":[("s1",0.7),("s2",0.1),("s3",0.1),("s6",0.1)],
    "s3":[("s2",0.4),("s3",0.2),("s7",0.4)],
    "s4":[("s4",0.1),("X",0.7),("s5",0.1),("s8",0.1)],
    "s5":[("s4",0.4),("s1",0.4),("s6",0.1),("s9",0.1)],
    "s6":[("s5",0.4),("s2",0.1),("s7",0.1),("s10",0.4)],
    "s7":[("s6",0.1),("s3",0.1),("s7",0.1),("s11",0.7)],
    "s8":[("s8",0.1),("s4",0.7),("s9",0.1),("s12",0.1)],
    "s9":[("s8",0.1),("s5",0.4),("s10",0.4),("s13",0.1)],
    "s10":[("s9",0.1),("s6",0.1),("s11",0.4),("s14",0.4)],
    "s11":[("s10",0.1),("s7",0.1),("s11",0.1),("X",0.7)],
    "s12":[("s12",0.2),("s8",0.4),("s13",0.4)],
    "s13":[("s12",0.1),("s9",0.1),("s14",0.7),("s13",0.1)],
    "s14":[("s13",0.1),("s10",0.1),("X",0.7),("s14",0.1)]
}

# Imports

In [2]:
from numpy.random import choice

# Grid World Simulator

In [3]:
class GridWorld:
    def __init__(self, policy_dict, terminal_state_key = "X"):
        self.policy = policy_dict
        self.non_terminal_states_cnt = len(self.policy)
        self.terminal_state_key = terminal_state_key
        
    @staticmethod
    def generate_reward(episode_seq):
        # All non-terminal states get reward 1 and terminal states get reward 0; gamma (discount factor) = 1
        return list(zip(episode_seq, list(range((-1)*(len(episode_seq)-2),1))))
        
    def generate_episode(self, reward = True):
        
        # generate s0 with equi-probability among 14 states
        current_state = f"s{choice(range(1,self.non_terminal_states_cnt+1), 1)[0]}"
        
        to_return = [current_state]
        while current_state != self.terminal_state_key:
            next_states_prob = list(zip(*self.policy[current_state]))
            current_state = choice(a=next_states_prob[0], p=next_states_prob[1], size=1)[0]
            to_return.append(current_state)
        if reward:
            return self.generate_reward(to_return)
        return to_return 
    
    def get_states(self):
        return list(self.policy.keys())

# Monte Carlo Helper

In [4]:
class MonteCarlo():
    def __init__(self, states, first_visit=True):
        self.states = states
        self.v_pi_with_cnt = {}
        self.reset_state_v_pi()
        self.first_visit = first_visit
        
    def reset_state_v_pi(self):
        for state in self.states:
            self.v_pi_with_cnt[state] = (None, 0)
            
    @staticmethod
    def retain_first_visits(state_reward_list):
        seen = set()
        return [(state, reward) for state, reward in state_reward_list
                if not (state in seen or seen.add(state))]
            
    def update_vpi(self, state_reward_list):
        if self.first_visit:
            state_reward_list = self.retain_first_visits(state_reward_list)
        for state, reward in state_reward_list:
            current_v_pi, cnt = self.v_pi_with_cnt[state]
            if current_v_pi is None:
                current_v_pi = 0
            cnt += 1
            self.v_pi_with_cnt[state] = (current_v_pi+((reward - current_v_pi)/cnt), cnt) 
            
    def return_v_pi(self):
        return [(state,self.v_pi_with_cnt[state][0]) for state in self.states]
    
    def __str__(self):
        to_return = "0".ljust(9," ")
        print_cnt = 1
        for state, v_pi in self.return_v_pi():
            prefix = "\n" if print_cnt%4 == 0 else ""
            to_return += (f"{prefix}{round(v_pi,3)}".ljust(10," "))
            print_cnt += 1
        return to_return+"0".ljust(9," ")
    

# Experiment

In [5]:
grid_world = GridWorld(policy)  # Policy from first section

In [6]:
episodes_cnt = 1000

In [7]:
first_visit_exp = MonteCarlo(grid_world.get_states(), first_visit=True)
every_visit_exp = MonteCarlo(grid_world.get_states(), first_visit=False)

In [8]:
for _ in range(episodes_cnt):
    episode = grid_world.generate_episode(reward=True)  # Generate episode
    first_visit_exp.update_vpi(episode)  # Update Vpi in first-visit
    every_visit_exp.update_vpi(episode)  # Update Vpi in every-visit

In [9]:
print("Result of first_visit_method:")
print(first_visit_exp)

Result of first_visit_method:
0        -0.805    -2.507    -3.458    
-0.997   -2.52     -3.263    -2.603    
-2.573   -3.382    -2.481    -1.099    
-3.833   -2.695    -1.007    0        


In [10]:
print("Result of every_visit_method:")
print(every_visit_exp)

Result of every_visit_method:
0        -0.856    -2.434    -3.312    
-1.016   -2.471    -3.243    -2.469    
-2.536   -3.388    -2.433    -1.052    
-3.73    -2.616    -0.976    0        
